In [1]:
from core.ConfigConnection import ConfigConnection
from core.Backup import Backup
from core.Tools import Tools

import mysql.connector

if __name__ == "__main__":
    config = ConfigConnection(path="./config.ini")
    connection = mysql.connector.connect(**config.getConfig())
    cursor = connection.cursor()
    tools = Tools()
    
    sql_file = tools.readFile('SQL', 'dml-web-scraping.sql')
    sql_script = ''.join(sql_file)

    sql_commands = sql_script.split(';')

    cursor = connection.cursor()
    
    try:
        # Ejecuta cada comando SQL individual
        for command in sql_commands:
            try:
                if command.strip():
                    cursor.execute(command)
            except mysql.connector.Error as err:
                print(f"Error executing SQL command: {err}")
           
        print("Script SQL ejecutado exitosamente.")
    except Exception as e:
        print("Error al ejecutar el script SQL:", e)
        connection.rollback()
    finally:
        cursor.close()
        connection.close()

Script SQL ejecutado exitosamente.


In [2]:
import configparser
import mysql.connector

fig = ConfigConnection(path="./config.ini")
connection = mysql.connector.connect(**config.getConfig())
cursor = connection.cursor()

try:
    # Ejecutar la consulta SHOW DATABASES
    cursor.execute("SHOW DATABASES")

    # Recorrer los resultados y mostrar los nombres de las bases de datos
    print("Bases de datos disponibles:")
    for (database,) in cursor:
        print(database)

except Exception as e:
    print("Error al ejecutar la consulta:", e)
finally:
    # Cerrar el cursor y la conexión
    cursor.close()
    connection.close()

Bases de datos disponibles:
IIDDBDWorldFactbook
IIDDBDsmartwatch
information_schema
mysql
performance_schema
sys


In [3]:
tools = Tools()

json_data = tools.readFileJson("data", "world-factbook.json")

db_config = {
    "host": "localhost",
    "user": "admin",
    "password": "admin",
    "database": "IIDDBDWorldFactbook"
}

connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()

# cursor.execute('SHOW TABLES;')

# print("Bases de datos disponibles:")
# for (tables,) in cursor:
#     print(tables)

for country, info in json_data.items():
    try:

        # Inserta en la tabla Countries
        cursor.execute("INSERT INTO Countries (tex_name, dec_population, dec_population_growth_percent, dec_area_km_sq, tex_map_references, int_coastline_km_sq, int_max_elevation_km, int_min_elevation_km, dec_population_density) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)",
                       (info['countrie'], 
                        float(info['population']), 
                        float(info['Population growth rate']) if info['Population growth rate'] != '' and info['Population growth rate'] != 'NA' else 0, 
                        float(info['area']) if info['area'] != '' else 0, 
                        info['Map references'] if info['Map references'] != 'france' else 'Europe', 
                        float(info['Coastline']), 
                        float(info['elevation highest point']) if info['elevation highest point'] != '' else 0, 
                        float(info['elevation lowest point']) if info['elevation lowest point'] != '' else 0, 
                        float(info['population']) / float(info['area']) if info['area'] != '' and float(info['area']) != 0 else 0
                       ))

#         # Inserta en la tabla Education (usando id_countries_fk como llave foránea)
#         cursor.execute("INSERT INTO Education (id_countries_fk, int_school_life_time, dec_education_cost_percent_GDP) VALUES (LAST_INSERT_ID(), %s, %s)",
#                        (info['education']['school life expectancy'], info['education']['Education expenditures']))
        
#         # Inserta en la tabla Economy (usando id_countries_fk como llave foránea)
#         cursor.execute("INSERT INTO Economy (id_countries_fk, dec_GDP_general_billion_of_dollars, dec_GDP_contribution_agriculture_percent, dec_GDP_contribution_industry_percent, dec_GDP_contribution_service_percent) VALUES (LAST_INSERT_ID(), %s, %s, %s, %s)",
#                        (info['economy']['GDP'], info['economy']['GDP_contribution_agriculture'], info['economy']['GDP_contribution_industry'], info['economy']['GDP_contribution_services']))

#         # Inserta en la tabla Health (usando id_countries_fk como llave foránea)
#         cursor.execute("INSERT INTO Health (id_countries_fk, int_life_expectancy_years, int_infant_mortality_deathover1000births) VALUES (LAST_INSERT_ID(), %s, %s)",
#                        (info['health']['life expectancy at birth'], info['health']['infant mortality rate']))

#         # Inserta en la tabla HealthProblems (usando id_health_fk como llave foránea)
#         cursor.execute("INSERT INTO HealhtProblems (id_health_fk, dec_health_expenses_PercenteGDP, int_physicians_over1000population, int_hospitalbeds_density_bedsover1000population) VALUES (LAST_INSERT_ID(), %s, %s, %s)",
#                        (info['health']['Health problems']['Current health expenditure'], info['health']['Health problems']['Physicians density'], info['health']['Health problems']['Hospital bed density']))

        # Realiza commit para cada país
        connection.commit()

    except mysql.connector.Error as err:
        print(f"Error inserting data for {info['countrie']}: {err}")

#Cierra el cursor y la conexión
cursor.close()
connection.close()

In [21]:
#                 print("Countries")
#                 print(info['countrie'], float(info['population']), 
#                       float(info['Population growth rate']) if info['Population growth rate'] != '' and info['Population growth rate'] != 'NA' else 0, 
#                       float(info['area']) if info['area'] != '' else 0, info['Map references'], info['Coastline'], info['elevation highest point'], info['elevation lowest point'], float(info['population']) / float(info['area']) if info['area'] != '' and float(info['area']) != 0 else 0)
#                 print("Economy")
#                 print(info['economy']['GDP'], info['economy']['GDP_contribution_agriculture'], info['economy']['GDP_contribution_industry'], info['economy']['GDP_contribution_services'])
#                 print("Education")
#                 print(info['education']['school life expectancy'], info['education']['Education expenditures'])
#                 print("Health")
#                 print(info['health']['life expectancy at birth'], info['health']['infant mortality rate'])
#                 print("HealthProblems")
#                 print(info['health']['Health problems']['Current health expenditure'], info['health']['Health problems']['Physicians density'], info['health']['Health problems']['Hospital bed density'])
#                 print()


85.38988312962798


SyntaxError: invalid syntax (1919572083.py, line 1)